In [ ]:
import pandas as pd
from datasets import Dataset
from datasets import load_dataset, load_from_disk, concatenate_datasets
from pathlib import Path
from tqdm import tqdm
import polars as pl
import numpy as np
from joblib import Parallel, delayed
import torch
from torch.utils.data.sampler import BatchSampler, RandomSampler

In [ ]:
#function that reduces the size of pandas dataframe
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# PATH_DATAST = Path("../data")
# train_meta = pl.scan_parquet(PATH_DATAST / "train_meta.parquet")
# geometry = pd.read_csv(PATH_DATAST / "sensor_geometry.csv")
# geometry["sensor_id"] = geometry["sensor_id"].astype("int16")
# geometry = pl.from_pandas(geometry).lazy()


# def batch_to_hf(batch_id, store_dist=Path("../data/hf_cashe")):

#     fn = PATH_DATAST / "train" / f"batch_{batch_id}.parquet"
#     train = pl.scan_parquet(fn)
#     train = (
#         train.join(geometry, on="sensor_id")
#         .groupby("event_id")
#         .agg([pl.all()])
#         .join(train_meta, on="event_id")
#         .collect()
#     )
#     train = train.to_pandas()[
#         [
#             "event_id",
#             "sensor_id",
#             "time",
#             "charge",
#             "auxiliary",
#             "x",
#             "y",
#             "z",
#             "azimuth",
#             "zenith",
#         ]
#     ]
#     # train = reduce_mem_usage(train)
#     dataset = Dataset.from_pandas(train)
#     dataset.save_to_disk(store_dist / fn.name)


In [ ]:
#get all the batches sorted 
# batches = sorted([int(x.stem.split("_")[1]) for x in (PATH_DATAST / "train").glob("batch_*.parquet")])

In [ ]:
# for i in tqdm(batches):
#     batch_to_hf(i)

from pdb import set_trace

In [ ]:
import torch

In [ ]:
#raed all hugingface datasets and concatenate them into one

#save the dataset to disk with batch_id as name 
#dataset.save_to_disk(PATH_DATAST / "hf_cashe" / "dataset_0-20")


In [ ]:
from copy import deepcopy


class HuggingFaceDataset(torch.utils.data.Dataset):
    def __init__(self, ds, max_events=100):
        self.ds = ds
        self.max_events = max_events

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]

        event = pd.DataFrame(item)[
            [
                "time",
                "charge",
                "auxiliary",
                "x",
                "y",
                "z",
            ]
        ].astype(np.float32)
        if self.max_events:
            event = event[: self.max_events]
        event["time"] /= event["time"].max()
        event[["x", "y", "z"]] /= 500
        event["charge"] = np.log10(event["charge"])

        event = event.values
        mask = np.ones(len(event), dtype=bool)
        label = np.array([item["azimuth"], item["zenith"]], dtype=np.float32)

        batch = deepcopy(
            {
                "event": torch.tensor(event),
                "mask": torch.tensor(mask),
                "label": torch.tensor(label),
            }
        )
        return batch


def collate_fn(batch):
    event = [x["event"] for x in batch]
    mask = [x["mask"] for x in batch]
    label = [x["label"] for x in batch]

    event = torch.nn.utils.rnn.pad_sequence(event, batch_first=True)
    mask = torch.nn.utils.rnn.pad_sequence(mask, batch_first=True)
    batch = {"event": event, "mask": mask, "label": torch.concat(label)}
    return batch


In [ ]:
dataset_custom = HuggingFaceDataset(hf_datasets)

dl = torch.utils.data.DataLoader(
    dataset_custom,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn,
    num_workers=16, 
    batch_size = 256, 
    persistent_workers=True,
)


In [ ]:
for x in tqdm(dl):
    pass

In [ ]:
len(hf_datasets)

In [ ]:
24.